*Created by Petteri Nevavuori <<petteri.nevavuori@gmail.com>>.*

---

# Building a General Purpose Blockchain

In this notebook we'll build a general purpose centralized blockhain as the first step into the realm of blockchains, distributed ledgers and smart contracts. We'll showcase the fundamental aspects of block-related functionalities. The implementation follows closely on the premises of course Blockchain A-Z found in Udemy but not identically. This is the first module of the course. We will utilize the Python-based Flask web-server package to serve the general purpose blockchain. The development itself will be separated to several stages.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Building-the-Blockhain" data-toc-modified-id="1.-Building-the-Blockhain-1">1. Building the Blockhain</a></span></li><li><span><a href="#2.-Creating-the-Web-App" data-toc-modified-id="2.-Creating-the-Web-App-2">2. Creating the Web App</a></span><ul class="toc-item"><li><span><a href="#2.1-Testing-the-Blockchain-App" data-toc-modified-id="2.1-Testing-the-Blockchain-App-2.1">2.1 Testing the Blockchain App</a></span></li></ul></li><li><span><a href="#3.-Mining--Blocks" data-toc-modified-id="3.-Mining--Blocks-3">3. Mining  Blocks</a></span><ul class="toc-item"><li><span><a href="#3.1-Testing-the-Chain-Validity" data-toc-modified-id="3.1-Testing-the-Chain-Validity-3.1">3.1 Testing the Chain Validity</a></span></li></ul></li></ul></div>

## 1. Building the Blockhain

The first thing of course is to build the blockchain. This blockchain utilizes a [proof-of-work](https://en.wikipedia.org/wiki/Proof-of-work_system) (POW) to validate the blocks. Essentially the POW states a problem that requires work to solve it and a target level for acceptance of the proof. With Bitcoins this means iteratively brute-forcing through [nonces](https://en.wikipedia.org/wiki/Cryptographic_nonce) to find the [SHA-256 hash](https://en.wikipedia.org/wiki/SHA-2) with four leading zeros. The probability of selecting a proper hexadecimal string with of 64 characters randomly is:

In [1]:
print("{:.5f}%".format(int('0000'+'F'*60,16)/int('F'*64,16)*100))

0.00153%


Our implementation's `proof` of a block essentially equals the nonce. This means that to produce a valid proof a number of iterations have to be tried out.

In our blockchain the block-wise proofs are calculated with the proof values of the previous block. This means that *changing the a proof value mid-chain would require re-calculating i.e. re-mining the blocks* to have new proofs and thus re-calculating the block-wise hashes. Because our initial blockchain does not have any data associated with it just yet, linking the proofs and blocks separately serves as a safety measure to ensure the tampering of our distributed ledger would be nearly impossible.

In [2]:
import datetime
import hashlib
import json


class Blockchain:

    def __init__(self):
        self.chain = []
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        block = {
            'index': len(self.chain),
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
        }
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"
        return (hashlib
                .sha256(str(next_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['proof']

            if self.hash_proof(previous_proof, next_proof)[:4] != '0000':
                
                return False

            return True

## 2. Creating the Web App

We will use Flask to create an accessible chain that is able to handle GET and POST requests for our chain accordingly. The endpoints for our web app are:

- ``[\blocks] POST``: Mine a single Block and save it to the centralized chain.
- ``[\blocks] GET``: Validate and retrieve the full chain of Blocks.

We will also use a distinct class to easen up the use of the server in a manner which allows for non-blocking use of the notebook. Thus we run the server in a separate thread. We will also wrap the request calls as more easily called functions of the app. The ``BlockchainApp`` is usable as a context manager to ensure that the distinct thread is closed.

An important point to notice is that the ``Blockchain`` instance will live separately from the ``BlockchainApp``. This means that while the ``BlockchainApp`` is re-initialized over and over, the actual Flask server accesses the ``Blockchain`` instance initialized at the same time with the server. Thus, the Blocks mined are persisted until the Python kernel is restarted.

In [3]:
import threading
import requests
import time

from flask import Flask, request, jsonify
import werkzeug

app = Flask(__name__)
blockchain = Blockchain()


@app.route('/blocks', methods=['GET', 'POST'])
def blocks():

    if request.method == 'POST':

        prev_block = blockchain.get_previous_block()
        prev_hash = blockchain.hash_block(prev_block)
        prev_proof = prev_block['proof']

        proof = blockchain.proof_of_work(prev_proof)

        block = blockchain.create_block(proof, prev_hash)

        response = {
            'message': 'Congratulations, you just mined a Block!',
            'block': block
        }

        return (jsonify(response), 200)

    if request.method == 'GET':

        response = {
            'blockchain': blockchain.chain,
            'length': len(blockchain.chain),
        }

        if blockchain.is_chain_valid(blockchain.chain):

            response['message'] = 'Chain is valid.'
            return (jsonify(response), 200)

        else:

            response['message'] = 'Chain is not valid!'
            return (jsonify(response), 500)


@app.route('/shutdown')
def shutdown():

    request.environ.get('werkzeug.server.shutdown')()

    return jsonify({'message': 'Shutting down'}), 200


class BlockchainApp:

    def __init__(self):

        self.host = 'localhost'
        self.port = 5000
        self.host_url = 'http://{}:{}'.format(self.host, self.port)
        self.t = threading.Thread(
            target=app.run,
            kwargs={'host': self.host, 'port': self.port})

    def start(self):
        self.t.start()

    def stop(self):
        if self.t.is_alive():
            return requests.get('{}/shutdown'.format(self.host_url))

    def mine_block(self):
        return requests.post('{}/blocks'.format(self.host_url))

    def get_blockchain(self):
        return requests.get('{}/blocks'.format(self.host_url))

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

### 2.1 Testing the Blockchain App

Let's then give the app a test run!

In [4]:
with BlockchainApp() as blockchain_app:

    print(blockchain_app.__class__.__name__)

    response = blockchain_app.mine_block()
    print(json.dumps(response.json(), indent=2))

    response = blockchain_app.get_blockchain()
    print(json.dumps(response.json(), indent=2))

 * Running on http://localhost:5000/ (Press CTRL+C to quit)


BlockchainApp


127.0.0.1 - - [11/Apr/2018 10:00:41] "POST /blocks HTTP/1.1" 200 -


{
  "block": {
    "index": 1,
    "previous_hash": "f198e29bb5e45e308d25579fc8f52a36fea66ba22ca61d18e654125c04116340",
    "proof": 533,
    "timestamp": "2018-04-11 10:00:41.379830"
  },
  "message": "Congratulations, you just mined a Block!"
}


127.0.0.1 - - [11/Apr/2018 10:00:42] "GET /blocks HTTP/1.1" 200 -


{
  "blockchain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "timestamp": "2018-04-11 10:00:40.317214"
    },
    {
      "index": 1,
      "previous_hash": "f198e29bb5e45e308d25579fc8f52a36fea66ba22ca61d18e654125c04116340",
      "proof": 533,
      "timestamp": "2018-04-11 10:00:41.379830"
    }
  ],
  "length": 2,
  "message": "Chain is valid."
}


127.0.0.1 - - [11/Apr/2018 10:00:43] "GET /shutdown HTTP/1.1" 200 -


Works like a prayer!

## 3. Mining  Blocks

Then we'll mine several blocks. This means that we'll start the server and make a few requests to the `/blocks` with a POST request.

In [5]:
with BlockchainApp() as blockchain_app:

    for _ in range(4):

        response = blockchain_app.mine_block()
        print("Proof={}".format(response.json()['block']['proof']))

    response = blockchain_app.get_blockchain()
    print("Chain={}".format(json.dumps(response.json(), indent=2)))

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2018 10:00:44] "POST /blocks HTTP/1.1" 200 -


Proof=45293


127.0.0.1 - - [11/Apr/2018 10:00:45] "POST /blocks HTTP/1.1" 200 -


Proof=21391


127.0.0.1 - - [11/Apr/2018 10:00:46] "POST /blocks HTTP/1.1" 200 -


...

127.0.0.1 - - [11/Apr/2018 10:00:48] "POST /blocks HTTP/1.1" 200 -


Proof=48191


127.0.0.1 - - [11/Apr/2018 10:00:49] "GET /blocks HTTP/1.1" 200 -


Chain={
  "blockchain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "timestamp": "2018-04-11 10:00:40.317214"
    },
    {
      "index": 1,
      "previous_hash": "f198e29bb5e45e308d25579fc8f52a36fea66ba22ca61d18e654125c04116340",
      "proof": 533,
      "timestamp": "2018-04-11 10:00:41.379830"
    },
    {
      "index": 2,
      "previous_hash": "8e0369ebcfe420065d5b3394c2ece9b06185757005b305a2d3e7277055281860",
      "proof": 45293,
      "timestamp": "2018-04-11 10:00:44.678168"
    },
    {
      "index": 3,
      "previous_hash": "df340007864a5fbd2a2465a9d2ac25f183416b5564a61fda2e87eb44d1ca18fb",
      "proof": 21391,
      "timestamp": "2018-04-11 10:00:45.831691"
    },
    {
      "index": 4,
      "previous_hash": "04f6299127b6fb0fb659e5b0ad54b3e2fa5d43739ae13e72a31da9ec04c7a18a",
      "proof": 8018,
      "timestamp": "2018-04-11 10:00:46.882162"
    },
    {
      "index": 5,
      "previous_hash": "003a0b87af5ad8d94070f7632b12b90ff18c

127.0.0.1 - - [11/Apr/2018 10:00:50] "GET /shutdown HTTP/1.1" 200 -


If you compare the first two Blocks with the test run you can see that the hashes and proofs match. This intended, as already stated.

### 3.1 Testing the Chain Validity

We also want to see if changing the proof mid-chain breaks the chain. For this we must access the initialized ``Blockchain`` directly as the miniserver doesn't have a method for altering the chain - which is a wise starting point!

In [6]:
original = blockchain.chain[3]['proof']
blockchain.chain[1]['proof'] = 12345
print('Proofs: Original={}, Forged={}'.format(original,blockchain.chain[3]['proof']))

with BlockchainApp() as blockchain_app:
    
    response = blockchain_app.get_blockchain()
    print(json.dumps(response.json(), indent=2))


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


Proofs: Original=21391, Forged=21391


127.0.0.1 - - [11/Apr/2018 10:00:51] "GET /blocks HTTP/1.1" 500 -


{
  "blockchain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "timestamp": "2018-04-11 10:00:40.317214"
    },
    {
      "index": 1,
      "previous_hash": "f198e29bb5e45e308d25579fc8f52a36fea66ba22ca61d18e654125c04116340",
      "proof": 12345,
      "timestamp": "2018-04-11 10:00:41.379830"
    },
    {
      "index": 2,
      "previous_hash": "8e0369ebcfe420065d5b3394c2ece9b06185757005b305a2d3e7277055281860",
      "proof": 45293,
      "timestamp": "2018-04-11 10:00:44.678168"
    },
    {
      "index": 3,
      "previous_hash": "df340007864a5fbd2a2465a9d2ac25f183416b5564a61fda2e87eb44d1ca18fb",
      "proof": 21391,
      "timestamp": "2018-04-11 10:00:45.831691"
    },
    {
      "index": 4,
      "previous_hash": "04f6299127b6fb0fb659e5b0ad54b3e2fa5d43739ae13e72a31da9ec04c7a18a",
      "proof": 8018,
      "timestamp": "2018-04-11 10:00:46.882162"
    },
    {
      "index": 5,
      "previous_hash": "003a0b87af5ad8d94070f7632b12b90ff18c4271

127.0.0.1 - - [11/Apr/2018 10:00:52] "GET /shutdown HTTP/1.1" 200 -


And so we indeed have invalidated the chain! Let's yet see what the hash should've been for Block in index ``2``, as it would be the one invalidated by the forgery.

In [13]:
proof_1 = blockchain.chain[1]['proof']
proof_2 = blockchain.chain[2]['proof']

forged_hash = blockchain.hash_proof(proof_1, proof_2)

print("Hashes: \n\tOriginal =\t{}\n\tForged =\t{}".format(blockchain.chain[2]['previous_hash'],forged_hash))

Hashes: 
	Original =	8e0369ebcfe420065d5b3394c2ece9b06185757005b305a2d3e7277055281860
	Forged =	b3e1a78ae4b5cd947a6690f4a5ee9b3a71a13a2df00d77cbc4c904c9b2ac2deb


What a difference. 

**This concludes the building and mining of a general purpose centralized Blockchain.**